In [ ]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import pandas as pd
import numpy as np

#######################Pre data setting########################################
model = VGG16(weights='imagenet', include_top=False, input_shape=(484,645,3)) #build a vgg model

img_path ='/Users/kkoo_/Documents/Python/code/Final_project/datasets/micrograph/'
data_path='/Users/kkoo_/Documents/Python/code/Final_project/datasets/micrograph.csv'
data_rd=pd.read_csv(data_path,sep=',') # read .csv as table
dataset=np.array(data_rd)
tname=list(['spheroidite','network','pearlite','spheroidite+widmanstatten',
			'pearlite+spheroidite','pearlite+widmanstatten','martensite']) 

primary=dataset[:,9] #get labels
pics=dataset[:,1] #get image path

sph_train,net_train,pearl_train,sph_wid_train=[],[],[],[]
sph_test,net_test,pearl_test,sph_wid_test=[],[],[],[]
pearl_sph_test,pearl_wid_test,martebsite_test=[],[],[]
for i in range(len(primary)): #seperate pictures
	if primary[i]==tname[0]:
		if len(sph_train)>=100:sph_test.append(pics[i])
		else: sph_train.append(pics[i])		
	elif primary[i]==tname[1]:
		if len(net_train)>=100:net_test.append(pics[i])
		else: net_train.append(pics[i])		
	elif primary[i]==tname[2]:
		if len(pearl_train)>=100:pearl_test.append(pics[i])
		else: pearl_train.append(pics[i])		
	elif primary[i]==tname[3]:
		if len(sph_wid_train)>=60:sph_wid_test.append(pics[i])
		else: sph_wid_train.append(pics[i])
	elif primary[i]==tname[4]:
		pearl_sph_test.append(pics[i])
	elif primary[i]==tname[5]:
		pearl_wid_test.append(pics[i])
	elif primary[i]==tname[6]:
		martebsite_test.append(pics[i])

###############################################################################

def getfeature(int_layer): #get features
	dim=int_layer.shape
	num=dim[0]*dim[1]*dim[2]
	return np.einsum('ijkl->l',int_layer)/num

def img2feature(x,block_pool):	 #transform image to features
	print('.')
	intermediate_layer = Model(inputs=model.input,outputs=model.get_layer(block_pool).output)
	int_layer=intermediate_layer.predict(x)
	return getfeature(int_layer)	#get vector

def gettrainingdata(train): # get training data
	v1=[[] for i in range(len(train))]		
	v2=[[] for i in range(len(train))]	
	v3=[[] for i in range(len(train))]	
	v4=[[] for i in range(len(train))]	
	v5=[[] for i in range(len(train))]	
	for i in range(len(train)):
		print('Train picture',i+1)
		img_o = image.load_img(img_path+train[i]) #load image
		img=img_o.crop((0,0,645,484)) # crop image as 484*645
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)	
		v1[i]=img2feature(x,'block1_pool')
		v2[i]=img2feature(x,'block2_pool')
		v3[i]=img2feature(x,'block3_pool')
		v4[i]=img2feature(x,'block4_pool')
		v5[i]=img2feature(x,'block5_pool')
	return v1,v2,v3,v4,v5

#get five features from pictures
[sphv1,sphv2,sphv3,sphv4,sphv5]=gettrainingdata(sph_train)
[netv1,netv2,netv3,netv4,netv5]=gettrainingdata(net_train)
[pearlv1,pearlv2,pearlv3,pearlv4,pearlv5]=gettrainingdata(pearl_train)
[sph_widv1,sph_widv2,sph_widv3,sph_widv4,sph_widv5]=gettrainingdata(sph_wid_train)

# for the rest of the qustions, using  only layer 5. Therefore, new code for getting feature v5
def gettrainingdata_v5(train):
	v=[[] for i in range(len(train))]	
	for i in range(len(train)):
		print('Train picture',i+1)
		img_o = image.load_img(img_path+train[i]) #load image
		img=img_o.crop((0,0,645,484)) # crop image as 484*645
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)	
		v[i]=img2feature(x,'block5_pool')
	return v

sph_test_v5=gettrainingdata_v5(sph_test)
net_test_v5=gettrainingdata_v5(net_test)
pearl_test_v5=gettrainingdata_v5(pearl_test)
sph_wid_test_v5=gettrainingdata_v5(sph_wid_test)
pearl_sph_test_v5=gettrainingdata_v5(pearl_sph_test)
pearl_wid_test_v5=gettrainingdata_v5(pearl_wid_test)

martebsite_test_v5=gettrainingdata_v5(martebsite_test)
#------------------------------------------------------------------------------

sphtrain=np.array([sphv1,sphv2,sphv3,sphv4,sphv5])
nettrain=np.array([netv1,netv2,netv3,netv4,netv5])
pearltrain=np.array([pearlv1,pearlv2,pearlv3,pearlv4,pearlv5])
sph_widtrain=np.array([sph_widv1,sph_widv2,sph_widv3,sph_widv4,sph_widv5])
#############################---Question a---#####################################
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

def error_CV10(train_one,train_two): #Calculate 10-fold cross validation error
	error_vc=[]
	for i in range(5): 
		train1=train_one[i][:]
		train2=train_two[i][:]
		class1=np.ones(len(train1))	
		class2=np.zeros(len(train2))
		train_data = np.concatenate([train1,train2])
		classes=np.concatenate([class1,class2])
		clf = svm.SVC(kernel = 'rbf',C = 1)
		train_data,classes=shuffle(train_data,classes,random_state=0)
		train_data=list(train_data)
		classes=list(classes)
		scores = cross_val_score(clf,train_data,classes, cv=10)#, scoring='average_precision')
		cv_error = 1 - np.sum(scores)/10 
		error_vc.append(cv_error)	
	return error_vc
#--------------------------------Q(a) Ans
error_cv_sn=error_CV10(sphtrain,nettrain)#sph,net
error_cv_sp=error_CV10(sphtrain,pearltrain)#sph,pearl
error_cv_ssw=error_CV10(sphtrain,sph_widtrain)#sph,sph+wid
error_cv_np=error_CV10(nettrain,pearltrain)#net,pearl
error_cv_nsw=error_CV10(nettrain,sph_widtrain)#net,sph
error_cv_psw=error_CV10(pearltrain,sph_widtrain)#pearl,sph+wid
#############################---Question a end---#####################################

############################---Question b---######################################
from sklearn.multiclass import OneVsOneClassifier
#---- Pairwise Error One vs One-----
def PairwiseError(train1,train2,test1,test2):
	train_data=np.concatenate((train1,train2))
	class1_train,class2_train=np.zeros(len(train1)),np.ones(len(train2))
	train_data=list(train_data)
	classes_tr=np.concatenate((class1_train,class2_train))
	OvO=OneVsOneClassifier(svm.SVC())
	OvO.fit(train_data,classes_tr)
	
	test_data=np.concatenate((test1,test2))
	class1_test,class2_test=np.zeros(len(test1)),np.ones(len(test2))
	classes_te=np.concatenate((class1_test,class2_test))	
	error=1-OvO.score(test_data,classes_te)
	return error

error_pair_sn=PairwiseError(sphtrain[4][:],nettrain[4][:],sph_test_v5,net_test_v5)
error_pair_sp=PairwiseError(sphtrain[4][:],pearltrain[4][:],sph_test_v5,pearl_test_v5)
error_pair_ssw=PairwiseError(sphtrain[4][:],sph_widtrain[4][:],sph_test_v5,sph_wid_test_v5)
error_pair_np=PairwiseError(nettrain[4][:],pearltrain[4][:],net_test_v5,pearl_test_v5)
error_pair_nsw=PairwiseError(nettrain[4][:],sph_widtrain[4][:],net_test_v5,sph_wid_test_v5)
error_pair_psw=PairwiseError(pearltrain[4][:],sph_widtrain[4][:],pearl_test_v5,sph_wid_test_v5)
#------------------------------
############################Multi Classes One vs One###########################################
train_data=np.concatenate([sphtrain[4][:],nettrain[4][:],pearltrain[4][:],sph_widtrain[4][:]])
class1,class2,class3,class4=np.zeros(len(sphtrain[4][:])),np.ones(len(nettrain[4][:])),2*np.ones(len(pearltrain[4][:])),3*np.ones(len(sph_widtrain[4][:]))
classes=np.concatenate((class1,class2,class3,class4))
train_data=list(train_data)
MOvO=OneVsOneClassifier(svm.SVC())
MOvO.fit(train_data,classes)
##############################################################################################

error_ovo_train=1-MOvO.score(train_data,classes) # Multi classes one vs one apparent error

test_data=np.concatenate([sph_test_v5,net_test_v5,pearl_test_v5,sph_wid_test_v5])
class1,class2,class3,class4=np.zeros(len(sph_test_v5)),np.ones(len(net_test_v5)),2*np.ones(len(pearl_test_v5)),3*np.ones(len(sph_wid_test_v5))
classes_test=np.concatenate((class1,class2,class3,class4))
MOvO_error_test=1-MOvO.score(test_data,classes_test) # Multi classes one vs one test error

sph_test_error=1-MOvO.score(sph_test_v5,class1)  # using each testing data to test the error of the classifier.
net_test_error=1-MOvO.score(net_test_v5,class2) 
pearl_test_error=1-MOvO.score(pearl_test_v5,class3) 
sph_wid_test_error=1-MOvO.score(sph_wid_test_v5,class4) 

############################---Question c---######################################
def classclasify(predictor):
	class1,class2,class3,class4=0,0,0,0
	for i in range(len(predictor)):
		if predictor[i]==0: 	class1 += 1
		elif predictor[i]==1: class2 += 1
		elif predictor[i]==2: class3 += 1
		elif predictor[i]==3: class4 += 1
	return class1,class2,class3,class4

ps_predict=MOvO.predict(pearl_sph_test_v5)
pw_predict=MOvO.predict(pearl_wid_test_v5)

ps_classes=classclasify(ps_predict)
pw_classes=classclasify(pw_predict)


############################---Question d---######################################
def PairwiseClassifier(train1,train2,predictor):
	train_data=np.concatenate((train1,train2))
	class1_train,class2_train=np.zeros(len(train1)),np.ones(len(train2))
	train_data=list(train_data)
	classes_tr=np.concatenate((class1_train,class2_train))
	OvO=OneVsOneClassifier(svm.SVC())
	OvO.fit(train_data,classes_tr)
	return OvO.predict(predictor)

def PairClassClasify(predictor):
	class1,class2,class3,class4=0,0,0,0
	for i in range(len(predictor)):
		if predictor[i]==0: 	class1 += 1
		elif predictor[i]==1: class2 += 1
	return class1,class2

ps_predict_pair=PairwiseClassifier(sphtrain[4][:],pearltrain[4][:],pearl_sph_test_v5)
ps_predict_pair_classes=PairClassClasify(ps_predict_pair)

############################---Question e---######################################
mar_predict=MOvO.predict(martebsite_test_v5) # using multi lalbel to predict
mar_predict_class=classclasify(mar_predict)

